In [6]:
from pyBADA.bada4 import Bada4Aircraft, ARPM
import pyBADA.atmosphere as atm

# 1. 已有的Aircraft对象
AC = Bada4Aircraft(
    badaVersion="4.2",
    acName="A320-232",
    filePath="/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models",
)

# 2. 创建ARPM实例
arpm = ARPM(AC)

# 3. 设置参数 - 使用英尺作为高度单位，然后转换为米
altitude_feet = 1000 # 高度，单位英尺
altitude_meters = altitude_feet * 0.3048  # 转换为米
mass = 60000      # 飞机质量，单位千克
runway_elevation_feet = 0  # 跑道海拔高度，单位英尺
runway_elevation_meters = runway_elevation_feet * 0.3048  # 转换为米

# 4. 计算给定高度的大气参数
DeltaTemp = 0.0  # 与ISA温度的偏差
theta_val = atm.theta(altitude_meters, DeltaTemp)
delta_val = atm.delta(altitude_meters, DeltaTemp)

In [ ]:
from pyBADA.bada4 import Bada4Aircraft, ARPM
import pyBADA.atmosphere as atm
import numpy as np
import pandas as pd

# 1. Initialize the Aircraft object
AC = Bada4Aircraft(
    badaVersion="4.2",
    acName="A320-232",
    filePath="/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models",
)

# 2. Create ARPM instance
arpm = ARPM(AC)

# 3. Define the Mach number
mach = 0.76

# 4. Define altitude range (in feet) to calculate airspeeds
# From sea level to 40,000 feet with 5,000 feet intervals
altitudes_feet = np.arange(0, 45000, 5000)

# 5. Create a list to store results
results = []

# 6. Calculate airspeeds at each altitude
for alt_feet in altitudes_feet:
    # Convert altitude to meters
    alt_meters = alt_feet * 0.3048
    
    # Calculate atmospheric parameters
    DeltaTemp = 0.0  # No deviation from ISA
    # 方案1：继续使用库函数，但确保正确计算大气参数
    theta_val, delta_val, sigma_val = atm.atmosphereProperties(alt_meters, DeltaTemp)
    cas = atm.mach2Cas(mach, theta_val, delta_val, sigma_val)
    tas = atm.mach2Tas(mach, theta_val)

    # Convert to knots for easier interpretation
    tas_knots = tas * 1.94384
    cas_knots = cas * 1.94384
    
    # Store results
    results.append({
        'Altitude (ft)': alt_feet,
        'Mach': mach,
        'TAS (m/s)': tas,
        'CAS (m/s)': cas,
        'TAS (knots)': tas_knots,
        'CAS (knots)': cas_knots,
        'theta': theta_val,
        'delta': delta_val,
        'sigma': sigma_val
    })

# 7. Create a DataFrame for better visualization
df = pd.DataFrame(results)

# 8. Print the results
print(df[['Altitude (ft)', 'Mach', 'TAS (knots)', 'CAS (knots)']])

   Altitude (ft)  Mach  TAS (knots)  CAS (knots)
0              0  0.76   435.911592   502.722574
1           5000  0.76   435.911592   463.427153
2          10000  0.76   435.911592   425.316099
3          15000  0.76   435.911592   388.529309
4          20000  0.76   435.911592   353.195271
5          25000  0.76   435.911592   319.427953
6          30000  0.76   435.911592   287.324418
7          35000  0.76   435.911592   256.963248
8          40000  0.76   435.911592   228.786043
